# 评估指标(主要关注排序相关指标)
NDCG,MAP,MRR: 

[MRR vs MAP vs NDCG：具有排序意义的度量指标的可视化解释及使用场景分析](https://blog.csdn.net/u011984148/article/details/107133086)


## NDCG
归一化累计折损收益，核心是高度关联的结果在更靠前的位置时，指标会更高

### CG(cumulative gain)

一共推荐了p个item，$r_i$表示位置i的item相关读度，$gc_p = \sum_{i}^{p} r_i$


### GDG(Discounted CG)
$$
  DCG_{p} = \sum_{i=1}^{p} \frac{r_i}{log_2(i + 1)} = \sum_{i=1}^{p} \frac{2^{r_i} - 1}{log_2(i + 1)}
$$

公式上看，对位置进行惩罚，越考前的，对最终DCG影响更大



### NDCG
归一化的DCG，因为每次搜索or推荐的item数量是不一样的，所以，DCG这个累加值就不具有可比性，需要归一化
$$
nDCG_{p} = \frac{DCG_p}{IDCG_p}
$$

其中$IDCG_p$是理想状态(ideal)下的$DCG_p$,比如，系统给用户推荐了5个item，M1,M2,M3,M4,M5,他们的评分为：5,2,3,1,4

则：$GC_5=5+2+3+1+4=15$

$DCG_p = \frac{2^5 - 1}{log_2 2} + \frac{2^2 - 1}{log_2 3} + \frac{2^3 - 1}{log_2 4} + \frac{2^1 - 1}{log_2 5} + \frac{2^4 - 1}{log_2 6} = 42.63$ 

现在来计算$iDCG_p$,如果相关电影一共6部，用户给M6评分为3,那么推荐6个相关电影理想排序：M1,M5,M3,M6,M2,评分为5,4,3,3,2,计算的$$iDCG_P = \frac{2^5 - 1}{log_2 2} + \frac{2^4 - 1}{log_2 3} + \frac{2^3 - 1}{log_2 4} + \frac{2^3 - 1}{log_2 5} + \frac{2^2 - 1}{log_2 6}=52.78$$

于是$nDCG_p=0.81$

优点：充分考虑了不同等级的相关性值

缺点：反馈数据获取预处理并不容易


## mRR
平均排名倒数，即推荐列表用户评分最高物体排名位置倒数
$$
 MRR = \frac{1}{|u|}\sum_{u \in U} \frac{1}{best\_index_u}
$$

如use1：3,4,5 ，use2：5,3,4，则$MRR=\frac{1}{2}(1/3 + 1/1) = 0.667 $

缺点：只关注第一个指标


## mAP
平均精度的平均精度。

如搜索系统，关键词i，关联N个item，$p_{ij}$表示i的关联列表中第j个item是否与之相关，取值为0-不相关或1-相关。
$$
AP_i = \frac{1}{N} \sum_{j=1}^{N} \frac{\sum_{m=1}^{j}p_{im}}{j}
$$
如query1展现3个item：相关，不相关，相关，则$AP = \frac{1}{3}(1/1 + 1/2 + 2/3)$

而：
$$
 mAP = \frac{1}{|Q|} \sum_{q} AP_q
$$

Q为所有待评估query

缺点:只适合0/1这种相关性评估

# 关联推荐

# CF

## 相似度计算

1. consine：$\frac{\mathbf{x} \cdot \mathbf{y}}{||\mathbf{x}||||\mathbf{y}||}$,不稳定，如果x+1,则consine值则有变化（当然，CF中item从0开始编码，没有这个问题）
2. 欧式距离$||\mathbf{x} - \mathbf{y}||$
3. 皮尔逊相关系数：存在多个式子，$\frac{cov(\mathbf{x}, \mathbf{y})}{\sigma_x \sigma_y}=\frac{E((x-\overline{x})(y-\overline{y}))}{\sigma_x \sigma_y} = \frac{\sum (x - \overline{x})(y - \overline{y})}{\sqrt{\sum (x - \overline{x})^2} \sqrt{\sum (y - \overline{y})^2}}$，相当于做了中心化的consine相似度


## User-Based CF

use较少场景，计算用户相似度，基于相似用户推荐高评分item。**更偏重兴趣群体中的热门商品**。

时效性较低：用户有新item点击时，自身被推荐新item时效性较低。依赖与相似群体行为变化

冷启动角度：用户行为很少时，很难计算准确的相似用户，比较难进行个性化推荐。

新Item敏感性：新item被点击，容易被推荐到该群体其他用户

推荐理由（偏产品侧）：比较难以产生好的推荐解释



## Item-Based CF

item较少场景，计算item相似度，推荐相关item。**偏用户个性化，偏长尾推荐**

时效性较高：用户有点击行为，可以根据相似item给出新的推荐

冷启动角度：只要有点击，就能推荐

新item敏感性：需要更新全局item相似度矩阵

推荐理由：用户有实际点击行为



升级版：item-based cf + 基于item内容推荐+新热表


## Item-Based CF 一种简单实现（map/reduce or spark）

用户数据：used_id, item_id, score/clicked

相似度计算方法：consine

计算方法：核心是计算item共现矩阵

第一轮map/reduce: 输出<item_i, item_j> 1, 用户数据合并后输出贡献item，i==j时，为item出现次数。通常，item数量有限，item出现次数可以使用字典形式

第二轮map/reduce：累计<item_i, item_j>出现次数，并计算相似度:$\frac{item_{i} \bigcap item_{j}}{item_{i} \cdot item_{j}}$,分子表示2个item共现次数，分母表示两个item分别被use点击次数。

# YouTube DNN 推荐算法

In [9]:
from IPython.display import IFrame
IFrame('https://static.googleusercontent.com/media/research.google.com/zh-CN//pubs/archive/45530.pdf', width=950, height=450)

## 疑问点
1. 新视频推荐问题：论文中只是添加了一个“sample age”特征， Rank阶段引入基于内容以召回，还是召回模型小时级增量更新（增量更新会导致embedding变化，更新时推荐架构侧如何处理，以空间换效果？）
2. 热门视频问题：论文提到不对热门视频做降权。（很容易推荐毫不关心的垃圾视频）

3. 召回模块训练为何负采样（论文解释：遍历树中节点,需要根据两个类别簇的区别进行路径选择，类别簇内节点内容通常是无关联的）


## 系统构建
系统分为2部分：召回候选集（数百万视频召回数百视频），排序模块（数百候选集做精排）


### 召回模块
推荐问题转换为超大规模分类问题：$P(w_t=i|U,C) = \frac{e^{v_i,u}}{\sum_{j \in V}e^{v_j u}}$,根据上下文，在t时刻推荐视频$w_i$的概率，softmax多分类问题。

input：单独训练的video/search embedding向量拼接地域，人口统计学等信息

训练：负采样+cross-entropy

3层Relu激活函数的隐层（塔式结构）

output：最后一个隐藏层输出user的embedding向量，softmax的参数组成video embedding矩阵输入在线服务，此处用LSH算法（经过研究，用simhash可以-相当于每个向量维度为hash的一位，然后按simhash计算思路，当某一维度>0,则变为1,小于0则变为0,将embedding转换为一个N维hash值，汉明距离就是他们相似度。实际服务，将它转换为N个桶，根据最小相似度，召回结果）


#### 异构信号
深度学习的一个有点是可以轻松的将任意连续和分类值添加到模型中

新视频问题：引入sample age特征

搜索记录，人口统计信息等都可以添加作为input

#### label与context选择
1. 使用更广泛的数据
2. 为每个用户生产固定数量原本：避免热门用户影响太大
3. 抛弃序列信息：我们直接对过去的观看视频embedding进行了avg，不考虑时间远近等序列信息
4. 不对称共同浏览问题：主要是label生成时，采用**predict next**




### Ranking


#### 优化目标
平均观看时长
$$\frac{\sum T_i}{N - k} = E[T] \cdot \frac{1}{1 - \frac{k}{N}} = E[T] \cdot \frac{1}{1 - p} \approx E[T] \cdot \frac{1}{1 + p} \approx E[T]$$

> LR的$odds = e^{wx +b} = \frac{P_{pos}}{P_{neg}}$，$\sum_{i}^{k}T_i$表示正样例权值，$N-k$为负样例，且 $N \gg k$,其中$p = \frac{k}{N}$为点击率

因此在线上serving的inference阶段，**我们采用$e^x$作为激励函数，就是近似的估计期望的观看时长[参考odds与$e^{wx+b}$关系]**。



#### 特征工程
1. 难点：建模用户时序行为，并将他们与要排序的item关联：如点击频次等数量特征，最近一次浏览频道的时间特征等。观看不点击等负反馈数据。视频来源等召回模型的部分特征
2. 高维离散特征embedding到稠密向量：如query（神经网络对连续特征更适合）
3. 归一化连续特征：归一化方法对收敛很关键，推荐一种排序分位归一到[0,1]区间的方法，即$\bar{x} = \int_{-\infty }^{x}df$累计分位点。除此之外，我们还把归一化后的$\bar{x}$的$\sqrt{x}$与$x^2$作为网络输入，以期能使网络能够更容易得到特征的次线性（sub-linear）和（super-linear）超线性函数。

# 实现

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import random
import copy
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence

In [2]:
data_path = "../../../learnMLData/"
# UserID::Gender::Age::Occupation::Zip-code
u_names = ['user_id','gender','age','occupation','zip_code']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=u_names, engine='python')
rate_names = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rate_names, engine='python')
move_names = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=move_names, engine='python')
# 如果未设置on与left_index, right_index，则交集为key
data0 = pd.merge(pd.merge(ratings,movies),user)

In [3]:
data = data0[0:10000]
data[0:3]

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip_code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067


In [4]:
data.columns

Index(['user_id', 'movie_id', 'rating', 'timestamp', 'title', 'genres',
       'gender', 'age', 'occupation', 'zip_code'],
      dtype='object')

# DSSM

In [10]:
IFrame('https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/cikm2013_DSSM_fullversion.pdf', width=950, height=450)

## Ref
https://juejin.cn/post/6844904168071626766